## RNA-Folding problem


L'RNA folding problem consiste nel predirre la struttura della molecola data una sequesza di nucleottidi. Questo è un problema molto noto alla comunità dei biologi ed è sempre stato studiato da molti punti di vista. Fino a pochi anni fa lo strumento utilizzato per risolverlo era la dynamic programming anche se comportava alcuna difficolta di formulazione del problema. In questo notebook illustrerò alcuni modelli (via via sempre più complicati) per la risoluzione dell'RNA folding problem usando la programmazione lineare intera (ILP). Con questo stumento si risolve il problema scrivendo modelli più semplice rispetto alla dynamic programming e anche più completi (anche se bisogna ammettere in media richiede più tempo computazionale).

# Un modello grezzo

Il primo modello che presento è il più semplice. L'RNA è formato da una catena di 4 nucleottidi: 'A', 'C', 'G', 'U'. Data una sequenza $S$, formata dalla ripetizone dei quattro caretteri siamo interessati a predirre la sua struttura bidimensionale. 
I nucleottidi si dividono in due categorie {'A', 'U'} e {'C', 'G'}, queste coppie sono dette complementari poichè gli elementi di una caregoriavtendono sempre a legarsi tra loro. 
Inoltre un legame tra due nucleottidi, poniamo in questo primo modello, non ne interseca un'altro come in figura. 

<img src= 'https://drive.google.com/uc?export=view&id=11k-y4T1rZFShYjAFsfvCC2fvzBj46HjM' width="20%">

Le coppie di nucleottidi legati tra loro che rispettano queste due condizioni le chiamiamo '**nested pair**'.

In questo modello vogliamo massimizzare la stabilità che è data dal numero di nested pair.
Ora creiamo in modello ILP che fomalizza quello detto fin qui.

### Variabile
Per prima cosa dobbiamo decidere una variabile decisonale per il problema. Introduciamo una variabile binaria (può assumere solo i valori {0,1}) $P(i,j)$ che poniamo uguale a 1 se e solo se il nucleottide i è legato (con una nested  pair) al nucleottide j.

###Vincoli
I vincoli necessari per imporre quelo detto fino ad ora sono 3:

* vogliamo che un nucleottide sia legato al più ad un'altro; allora introduciamo il vincolo:
> $\sum_{i>j}\ P(i,j) + \sum_{i<j}\ P(i,j) \leq 1 \quad \forall j \in S$

* vogliamo che P(i,j) è 1 se e solo se i,j sono complemetari cioè se i,j sono uguali a {'A','U'}, {'U','A'}, {'C','G'}, {'G','C'}.

* per ultimo vogliamo ammettere solo le nested pairs:
> $P(i,j) + P(k,h) \leq 1 \quad \forall i<k<j<h$ 

### Funzione obbiettivo
La funzione obbiettivo è:
> $max \sum_{i<j}\ P(i,j)$



Finalmente si può scrivere un primo codice

In [0]:
#importo pyomo e il solver necessario (glpk)

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

In [0]:
#importo il necessario da pyomo

from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, Set
from pyomo.environ import maximize, Binary, RangeSet, Param, ConstraintList, PositiveIntegers
import numpy as np

model = ConcreteModel()

#inserisco dei dati (fantocci)
data = [1,1,1,1,1,2,3,2,4,4,3,2,2,3,2,1,1,1,2,3]
D = {index+1 : value for index,value in enumerate(data)}
n = len(data)
#indices
model.I = RangeSet(1, n)
model.J = RangeSet(1, n)

#parameter
model.nucleot = Param(model.I, initialize = D)

#variable
model.P = Var(model.I, model.J, within = Binary, initialize = 0)

#constraint
model.onebond = Constraint(model.I, rule = lambda model, i: sum(model.P[i,j] + model.P[j,i] for j in model.J) <= 1)

def paired_expr(model, i, j):
    if model.nucleot[i] == model.nucleot[j]:
        return model.P[i,j] <= 0
    elif model.nucleot[i] == 1 and model.nucleot[j] == 2:
        return  model.P[i,j] <= 0
    elif model.nucleot[i] == 2 and model.nucleot[j] == 1:
        return  model.P[i,j] <= 0
    elif model.nucleot[i] == 1 and model.nucleot[j] == 3:
        return model.P[i,j] <= 0
    elif model.nucleot[i] == 3 and model.nucleot[j] == 1:
        return model.P[i,j] <= 0
    elif model.nucleot[i] == 2 and model.nucleot[j] == 4:
        return  model.P[i,j] <= 0
    elif model.nucleot[i] == 4 and model.nucleot[j] == 2:
        return  model.P[i,j] <= 0
    elif model.nucleot[i] == 3 and model.nucleot[j] == 4:
        return model.P[i,j] <= 0
    elif model.nucleot[i] == 4 and model.nucleot[j] == 3:
        return model.P[i,j] <= 0
    return model.P[i,j] <= 1

model.bond_exprs = ConstraintList()
for i in range(1,n+1):
    for j in range(1,n+1):
        model.bond_exprs.add(paired_expr(model,i,j))

def no_cross_rule(model, i, j, h, k):
    return model.P[i,j] + model.P[h,k] <= 1

model.no_cross_pair = ConstraintList()
for i in range(1,n+1):
    for j in range(i,n+1):
        for h in range(1,i):
            for k in range(i+1,j):
                model.no_cross_pair.add(no_cross_rule(model,i,j,h,k))
                model.no_cross_pair.add(no_cross_rule(model,j,i,h,k))
                model.no_cross_pair.add(no_cross_rule(model,i,j,k,h))
                model.no_cross_pair.add(no_cross_rule(model,j,i,k,h))

#obj function
model.obj = Objective(expr = sum(model.P[i,j] for i in model.I for j in model.J), sense = maximize)


In [0]:
#risolvo il modello

sol = SolverFactory('glpk').solve(model)
sol_json = sol.json_repn()

if sol_json['Solver'][0]['Status'] != 'ok':
    print("Problem unsolved")
if sol_json['Solver'][0]['Termination condition'] != 'optimal':
    print("Problem unsolved")

#printo gli i,j che sono legati tra loro e il valore della funzione obbiettivo

for i in range(1,n+1):
    for j in range(1,n+1):
        if model.P[i,j] == 1:
            print(i,j)

1 10
6 7
9 2
11 12
13 14
15 20


# Secondo modello

Alcune estensioni utili che modelizza quello che succede quando una molecola di RNA si assembla sono:
* settare $P(i,j) = 0$ se i e j sono due nucleottidi consecutivi
* l'ultima modifica che facciamo è più profonda. Diciamo che due coppie sono '**stacked pairs**' se sono consecutive, cioè $P(i,j) = 1$ e $P(i+1,j-1) = 1$. in questo caso i < i +1 < j-1 < j è detto uno stack quartet. Allora definiamo una '**stack**' l'insime di 2 o più stacked pairs.
Questa struttura è importante perchè gli stack creano (se si guarda la struttura in due dimensioni) creano degli '**stem**' cioè delle filamenti dritti nella struttura. In figura le parti verdi, blu e rosse.

> <img src= 'https://drive.google.com/uc?export=view&id=1X7TQTHwMOFiIw26WlRhiR7KErGvCTnfH' width="50%">

È importante incoraggiare questa struttura poichè è molto stabile (biologicamanete parlando).
Ora vediamo come creare il modello ILP corrispondente.

### Variabili
Per imporre l'ultima condizione è necessario introdurre una nuova variabile oltre a $P$. Perciò definiamo $Q(i,j)$ la nostra nuova variabile anch'essa binaria che indica se la coppia $(i,j)$ appartiene ad una stack quartet oppure no.

###Matrice dei pesi
Un'altra cosa da calcolare per questa estensione sono i pesi. Poichè non posso metterli come una variabile del problema, se no la funzione obbiettivo diventa quadratica, devo calcolarli a priori. Infatti si crea una matrice $C$ di pesi che avrà nella posizione $i,j$ il peso corrispondente al legame che si potrebbe creare tra $i$ e $j$. Poi selezionerò nella funzione obbiettivo solo i pesi tra nucleottidi effettivamente legati tra loro.

###Vincoli
I vincoli necessari sono:
* imporre $P(i,j) = 0$ se $i$ e $j$ sono consecutivi:
> $P(i,i+1) = 0 \quad \forall i \in S$ 
* i vincoli sulla variabile $Q$:
  1. $\quad  P(i,j) +P(i+1,j-1)- Q(i,j) \leq 1 \quad \forall i < j$ 
  2. $\quad  2*Q(i,j) -P(i,j)-P(i+1,j-1) \leq 0 \quad \forall i< j$

Si noti che questi due vincoli sono uno l'inverso dell'altro. Infatti il primo forza $Q(i,j) = 1$ se $P(i,j) = P(i+1,j-1) = 1$; mentre il secondo impone $P(i,j )  = P(i+1,j-1) = 1 $ se $Q(i,j)=  1$


### Funzione obbiettivo
La funzione obbiettivo per questo modello è:
> $max \sum_{i<j}\ [C(i,j)*P(i,j)+Q(i,j)]$

Un'altra funzione obbiettivo che risulta addirittura più accurata biologicamente e incredibilmente riduce anche il tempo di risoluzione è:
> $max \sum_{i<j}\ Q(i,j)$





In [0]:
model = ConcreteModel()

#inserisco dei dati (fantocci)
data = [1,1,1,1,1,2,3,2,4,4,3,2,2,3,2,1,1,1,2,3]
D = {index+1 : value for index,value in enumerate(data)}
n = len(data)

#indices
model.I = RangeSet(1, n)
model.J = RangeSet(1, n)

#parameter
model.nucleot = Param(model.I, initialize = D)

#variable, aggiungo Q
model.P = Var(model.I, model.J, within = Binary, initialize = 0)
model.Q = Var(model.I, model.J, within = Binary, initialize = 0)

#build matrix C
def matrice_costi(data):
    n = len(data)
    C = np.ones((n,n))
    for i in range(n):
        for j in range(n):
          if (data[i]== 2 and data[j]== 3) or (data[i] == 3 and data[j] == 2):
            C[i,j] = 1.2 
          if not ((data[i] == 1 and data[j] == 4) or (data[i] == 4 and data[j] == 1)):
           if not ((data[i] == 2 and data[j] == 3) or (data[i] == 3 and data[j] == 2)):
            C[i,j] = 0.7
    return C

C = matrice_costi(data)

#constraint
model.onebond = Constraint(model.I, rule = lambda model, i: sum(model.P[i,j] + model.P[j,i] for j in model.J) <= 1)

def stack_pairs1(model, i, j):
    return model.P[i,j] + model.P[i+1,j-1] - model.Q[i,j] <= 1

def stack_pairs2(model, i, j):
    return 2*model.Q[i,j] - model.P[i,j] - model.P[i+1,j-1] <= 0

model.stack_pairs = ConstraintList()
for i in range(1,n+1):
    for j in range(i+1,n+1):
        model.stack_pairs.add(stack_pairs1(model,i,j))
        model.stack_pairs.add(stack_pairs2(model,i,j))

def no_cross_rule(model, i, j, h, k):
    return model.P[i,j] + model.P[h,k] <= 1

model.no_cross_pair = ConstraintList()
for i in range(1,n+1):
    for j in range(i,n+1):
        for h in range(1,i):
            for k in range(i+1,j):
                model.no_cross_pair.add(no_cross_rule(model,i,j,h,k))
                model.no_cross_pair.add(no_cross_rule(model,j,i,h,k))
                model.no_cross_pair.add(no_cross_rule(model,i,j,k,h))
                model.no_cross_pair.add(no_cross_rule(model,j,i,k,h))

def no_consecutive_rule(model,i):
  return model.P[i,i+1] <= 0

model.no_consecutive_pair = ConstraintList()
for i in range(1,n):
  model.no_consecutive_pair.add(no_consecutive_rule(model,i))

#obj function
model.obj = Objective(expr = sum(C[i-1,j-1]*model.P[i,j] + model.Q[i,j] for i in model.I for j in model.J), sense = maximize)

In [0]:
#risolvo il modello

sol = SolverFactory('glpk').solve(model)
sol_json = sol.json_repn()

if sol_json['Solver'][0]['Status'] != 'ok':
    print("Problem unsolved")
if sol_json['Solver'][0]['Termination condition'] != 'optimal':
    print("Problem unsolved")

#printo gli i,j che sono legati tra loro e il valore della funzione obbiettivo
for i in range(1,n+1):
    for j in range(1,n+1):
        if model.P[i,j] == 1:
            print(i,j)

1 5
2 4
6 20
7 19
8 18
9 17
10 16
11 15
12 14


In [0]:
#provo con l'altra funzione obbiettivo
model.obj = Objective(expr = sum(model.Q[i,j] for i in model.I for j in model.J), sense = maximize)

    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [0]:
#risolvo il modello

sol = SolverFactory('glpk').solve(model)
sol_json = sol.json_repn()

if sol_json['Solver'][0]['Status'] != 'ok':
    print("Problem unsolved")
if sol_json['Solver'][0]['Termination condition'] != 'optimal':
    print("Problem unsolved")

#printo gli i,j che sono legati tra loro e il valore della funzione obbiettivo
for i in range(1,n+1):
    for j in range(1,n+1):
        if model.P[i,j] == 1:
            print(i,j)


1 20
2 19
3 18
4 17
5 16
6 15
7 14
8 13
9 12
Q : Size=400, Index=Q_index
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1) :     0 :   1.0 :     1 : False : False : Binary
      (1, 2) :     0 :   0.0 :     1 : False : False : Binary
      (1, 3) :     0 :   0.0 :     1 : False : False : Binary
      (1, 4) :     0 :   0.0 :     1 : False : False : Binary
      (1, 5) :     0 :   0.0 :     1 : False : False : Binary
      (1, 6) :     0 :   0.0 :     1 : False : False : Binary
      (1, 7) :     0 :   0.0 :     1 : False : False : Binary
      (1, 8) :     0 :   0.0 :     1 : False : False : Binary
      (1, 9) :     0 :   0.0 :     1 : False : False : Binary
     (1, 10) :     0 :   0.0 :     1 : False : False : Binary
     (1, 11) :     0 :   0.0 :     1 : False : False : Binary
     (1, 12) :     0 :   0.0 :     1 : False : False : Binary
     (1, 13) :     0 :   0.0 :     1 : False : False : Binary
     (1, 14) :     0 :   0.0 :     1 : False : False : Bina

# Modello completo

È possibile fare molte altre modifiche al codice; le librerie specializzate nel trattare il probelma del folding dell'RNA hanno più di 200 diversi parametri a seconda della situazione.
L'ultima modifica che applicherò al modello riguarda il numero di stack all'interno della struttura dell'RNA. In particolare modificherò il codice affinchè ci siano il minor numero di stack (e quindi più lunghi e più stabili) nella folding che predirrò.

###Variabile
Aggiungo anche in questo caso una nuova variabile $F(i,j)$, anch'essa binaria, che pongo uguale a 1 se e solo se $(i,j)$ è la prima pairs di una stack. 

### Vincoli
dobbiamo aggiungere i vincoli su $F(i,j)$ similmente a come abbiamo fatto per $Q$:
1. $\quad  Q(i,j) -Q(i-1,j+1)- F(i,j) \leq 0 \quad \forall i < j$ 
2. $\quad  2*F(i,j) -Q(i,j)-Q(i-1,j+1) \leq 0 \quad \forall i< j$
il primo impone che se $Q(i-1,j+1 ) = 1$ allora $Q(i,j) = 1 \iff F(i,j)= 1$ . Mentre il secondo è l'implicazione opposta: se $F(i,j) = 1$ allora $Q(i,j) = 1$ e $Q(i-1,j+1) = 0$

###Funzione obbiettivo
Per minimizzare il numero di stack basta sottrarre alla funzione obbiettivo precedente la variabile $F$:
> $max \sum_{i<j}\ [C(i,j)*P(i,j)+Q(i,j)-F(i,j)]$

In [0]:
model = ConcreteModel()

#inserisco dei dati (fantocci)
data = [1,1,1,1,1,2,3,2,4,4,3,2,2,3,2,1,1,1,2,3]
D = {index+1 : value for index,value in enumerate(data)}
n = len(data)

#indices
model.I = RangeSet(1, n)
model.J = RangeSet(1, n)

#parameter
model.nucleot = Param(model.I, initialize = D)

#variable, aggiungo Q
model.P = Var(model.I, model.J, within = Binary, initialize = 0)
model.Q = Var(model.I, model.J, within = Binary, initialize = 0)
model.F = Var(model.I, model.J, within = Binary, initialize = 0)

#build matrix C
def matrice_costi(data):
    n = len(data)
    C = np.ones((n,n))
    for i in range(n):
        for j in range(n):
          if (data[i]== 2 and data[j]== 3) or (data[i] == 3 and data[j] == 2):
            C[i,j] = 1.2 
          if not ((data[i] == 1 and data[j] == 4) or (data[i] == 4 and data[j] == 1)):
           if not ((data[i] == 2 and data[j] == 3) or (data[i] == 3 and data[j] == 2)):
            C[i,j] = 0.7
    return C

C = matrice_costi(data)

#constraint
model.onebond = Constraint(model.I, rule = lambda model, i: sum(model.P[i,j] + model.P[j,i] for j in model.J) <= 1)

def stack_pairs1(model, i, j):
    return model.P[i,j] + model.P[i+1,j-1] - model.Q[i,j] <= 1

def stack_pairs2(model, i, j):
    return 2*model.Q[i,j] - model.P[i,j] - model.P[i+1,j-1] <= 0

model.stack_pairs = ConstraintList()
for i in range(1,n+1):
    for j in range(i+1,n+1):
        model.stack_pairs.add(stack_pairs1(model,i,j))
        model.stack_pairs.add(stack_pairs2(model,i,j))

def no_cross_rule(model, i, j, h, k):
    return model.P[i,j] + model.P[h,k] <= 1

model.no_cross_pair = ConstraintList()
for i in range(1,n+1):
    for j in range(i,n+1):
        for h in range(1,i):
            for k in range(i+1,j):
                model.no_cross_pair.add(no_cross_rule(model,i,j,h,k))
                model.no_cross_pair.add(no_cross_rule(model,j,i,h,k))
                model.no_cross_pair.add(no_cross_rule(model,i,j,k,h))
                model.no_cross_pair.add(no_cross_rule(model,j,i,k,h))

def no_consecutive_rule(model,i):
  return model.P[i,i+1] <= 0

model.no_consecutive_pair = ConstraintList()
for i in range(1,n):
  model.no_consecutive_pair.add(no_consecutive_rule(model,i))

def F_rule1(model,i,j):
  return model.Q[i,j] - model.Q[i-1,j+1] - model.F[i,j] <= 0


def F_rule2(model,i,j):
  return 2*model.F[i,j] - model.Q[i,j] + model.Q[i-1,j+1] <= 0

model.F_rules = ConstraintList()
for i in range(2,n):
    for j in range(i,n):
        model.F_rules.add(F_rule1(model,i,j))
        model.F_rules.add(F_rule2(model,i,j))

#obj function
model.obj = Objective(expr = sum(C[i-1,j-1]*model.P[i,j] + model.Q[i,j] - model.F[i,j] for i in model.I for j in model.J), sense = maximize)

In [0]:
#risolvo il modello

sol = SolverFactory('glpk').solve(model)
sol_json = sol.json_repn()

if sol_json['Solver'][0]['Status'] != 'ok':
    print("Problem unsolved")
if sol_json['Solver'][0]['Termination condition'] != 'optimal':
    print("Problem unsolved")

#printo gli i,j che sono legati tra loro e il valore della funzione obbiettivo
for i in range(1,n+1):
    for j in range(1,n+1):
        if model.P[i,j] == 1:
            print(i,j)

1 10
4 3
6 5
8 7
9 2
12 11
14 19
16 15
18 17
20 13
